In [14]:
import os
import ctypes

dll_path = r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\openslide-bin-4.0.0.8-windows-x64\bin"
os.add_dll_directory(dll_path)  # Python 3.8 이상에서만 사용 가능

# 혹은 수동으로 로드 (fallback)
# ctypes.WinDLL(os.path.join(dll_path, "libopenslide-0.dll"))  (아직 실행 x)


<AddedDllDirectory('C:\\Users\\roobe\\PKG - HER2 tumor ROIs_v3\\openslide-bin-4.0.0.8-windows-x64\\bin')>

In [15]:
import pandas as pd

# 엑셀 파일 경로
file_path = r'C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_trastuzumab_response_cohort\her2.xlsx'

# 엑셀 파일 읽기
df = pd.read_excel(file_path)

# 데이터프레임 출력
print(df)


      Patient     Responder  Response
0   O09-03495  nonresponder  positive
1   O10-12717  nonresponder  positive
2   O14-02301  nonresponder  positive
3   O16-11870     responder  positive
4   O16-18464  nonresponder  positive
..        ...           ...       ...
80  S18-31022     responder  positive
81  S18-32412  nonresponder  positive
82  S13-07627     responder  positive
83  S13-08586  nonresponder  positive
84  S18-15441     responder  positive

[85 rows x 3 columns]


In [16]:
#용량 부족으로 기존에 전체 패치 나눴던 사진 삭제
import shutil
import os

patches_dir = r"C:\users\roobe\PKG - HER2 tumor ROIs_v3/Yale_trastuzumab_response_cohort/patches/test"

# 폴더 내 모든 파일과 폴더 삭제
for filename in os.listdir(patches_dir):
    file_path = os.path.join(patches_dir, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)  # 파일 또는 링크 삭제
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  # 하위 폴더 삭제
    except Exception as e:
        print(f"삭제 실패: {file_path}, 이유: {e}")

In [18]:
import shutil
import os

patches_dir = r"C:\users\roobe\PKG - HER2 tumor ROIs_v3\Yale_trastuzumab_response_cohort\patches"

# 폴더 내 모든 PNG 파일만 삭제
for filename in os.listdir(patches_dir):
    file_path = os.path.join(patches_dir, filename)
    try:
        if os.path.isfile(file_path) and filename.lower().endswith('.png'):
            os.unlink(file_path)  # PNG 파일만 삭제
    except Exception as e:
        print(f"삭제 실패: {file_path}, 이유: {e}")


삭제 실패: C:\users\roobe\PKG - HER2 tumor ROIs_v3\Yale_trastuzumab_response_cohort\patches\O16-11870_1510_3430.png, 이유: [WinError 32] 다른 프로세스가 파일을 사용 중이기 때문에 프로세스가 액세스 할 수 없습니다: 'C:\\users\\roobe\\PKG - HER2 tumor ROIs_v3\\Yale_trastuzumab_response_cohort\\patches\\O16-11870_1510_3430.png'


In [20]:


import os
import xml.etree.ElementTree as ET
import openslide
import numpy as np
from shapely.geometry import Point, Polygon 


def parse_tumor_polygons(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    tumor_polygons = []

    for annotation in root.findall(".//Annotation"):
        name = annotation.get("Name")
        if name != "Tumor":
            continue

        for region in annotation.findall(".//Region"):
            vertices = []
            for vertex in region.findall(".//Vertex"):
                x = float(vertex.get("X"))
                y = float(vertex.get("Y"))
                vertices.append((x, y))
            if vertices:
                tumor_polygons.append(vertices)

    return tumor_polygons


def extract_tumor_patches(svs_path, xml_path, output_dir, patch_size=512, level=0):
    # 슬라이드 열기
    slide = openslide.OpenSlide(svs_path)
    slide_name = os.path.splitext(os.path.basename(svs_path))[0]

    # XML 파일로부터 암으로 주석된 영역(폴리곤)들의 좌표를 가져옵
    tumor_polygons_coords = parse_tumor_polygons(xml_path)
    # shapely Polygon 객체 리스트로 변환->리스트 숫자값에서 폴리건 리스트로 변환시켜 공간 연산 등을 가능하게 함.
    tumor_polygons = [Polygon(coords) for coords in tumor_polygons_coords]

    # 암 영역 전체 bounding box (모든 폴리곤 포함해서 최소, 최대 좌표 구함)
    all_x = [pt[0] for poly in tumor_polygons_coords for pt in poly]
    all_y = [pt[1] for poly in tumor_polygons_coords for pt in poly]
    min_x, max_x = int(min(all_x)), int(max(all_x))#전체 암 영역이 어디까지인지 확인하기 위해, 모든 폴리곤의 x/y 좌표의 최소/최대값을 구함->이걸로 패치 만들 범위를 정함
    min_y, max_y = int(min(all_y)), int(max(all_y))

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for x in range(min_x, max_x, patch_size):
        for y in range(min_y, max_y, patch_size):
            center = Point(x + patch_size // 2, y + patch_size // 2)
            # 중심점이 암 폴리곤 내부에 있는지 체크
            inside_any_poly = any(poly.contains(center) for poly in tumor_polygons)
            if inside_any_poly:
                patch = slide.read_region((x, y), level, (patch_size, patch_size)).convert("RGB")
                patch_np = np.array(patch)
                # 배경 제거: 너무 밝으면 배경으로 판단 (필요하면 조정)
                if np.mean(patch_np) < 220:
                    filename = f"{slide_name}_{x}_{y}.png"
                    patch.save(os.path.join(output_dir, filename))
   

# 사용 예시
svs_dir = r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_trastuzumab_response_cohort\SVS"
xml_dir =r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_trastuzumab_response_cohort\Annotations"
output_dir = r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_trastuzumab_response_cohort\patches"
for file in os.listdir(svs_dir):
    if file.endswith('.svs'):
        svs_path = os.path.join(svs_dir, file)
        base_name = os.path.splitext(file)[0]
        xml_path = os.path.join(xml_dir, base_name + ".xml")

        if not os.path.exists(xml_path):
            print(f"❌ XML not found for {file}")
            continue

        try:
            print(f"✅ Processing {file}")
            extract_tumor_patches(svs_path, xml_path, output_dir, patch_size=512, level=0)
        except Exception as e:
            print(f" Error processing {file}: {e}")





✅ Processing O09-03495.svs
✅ Processing O10-12717.svs
✅ Processing O14-02301.svs
✅ Processing O16-11870.svs
✅ Processing O16-18464.svs
✅ Processing O17-01529.svs
✅ Processing O18-12772.svs
✅ Processing S08-31466.svs
✅ Processing S09-15833.svs
✅ Processing S09-19750.svs
✅ Processing S10-00647.svs
✅ Processing S10-09633.svs
✅ Processing S11-15267.svs
✅ Processing S11-24835.svs
✅ Processing S12-07776.svs
✅ Processing S12-22754.svs
✅ Processing S12-28303.svs
✅ Processing S12-35886.svs
✅ Processing S13-07627.svs
✅ Processing S13-08586.svs
✅ Processing S13-11381.svs
✅ Processing S13-15979.svs
✅ Processing S13-31494.svs
✅ Processing S13-33035.svs
✅ Processing S14-01774.svs
✅ Processing S14-04504.svs
✅ Processing S14-12226.svs
✅ Processing S14-12770.svs
✅ Processing S14-12928.svs
✅ Processing S14-13440.svs
✅ Processing S14-13994.svs
✅ Processing S14-15197.svs
✅ Processing S14-16118.svs
✅ Processing S15-00942.svs
✅ Processing S15-02790.svs
✅ Processing S15-03056.svs
✅ Processing S15-05781.svs
✅

In [22]:
import os
import shutil
import pandas as pd
import random

# 전제 설정
patch_dir = r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_trastuzumab_response_cohort\patches"
output_base = r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_trastuzumab_response_cohort\patches"
metadata_path = r'C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_trastuzumab_response_cohort\her2.xlsx'

# 엑셀 데이터 불러오기
df = pd.read_excel(metadata_path)

# 환자 목록 분리
patients = df['Patient'].unique().tolist()
random.seed(42)
random.shuffle(patients)
split_idx = int(len(patients) * 0.8)
train_patients = set(patients[:split_idx])
test_patients = set(patients[split_idx:])

# 전체 패치 파일 목록
all_patches = [f for f in os.listdir(patch_dir) if f.endswith('.png')]

# 파일 이동
for patch in all_patches:
    patient_id = patch.split('_')[0]
    if patient_id not in df['Patient'].values:
        continue

    label = df.loc[df['Patient'] == patient_id, 'Responder'].values[0]
    group = 'train2' if patient_id in train_patients else 'test2'

    src_path = os.path.join(patch_dir, patch)
    dst_path = os.path.join(output_base, group, label, patch)

    if not os.path.exists(src_path):
        print(f"파일 없음: {src_path}")
        continue

    os.makedirs(os.path.dirname(dst_path), exist_ok=True)  # 중간 폴더 생성

    try:
        shutil.move(src_path, dst_path)
        print(f"이동 완료: {patch}")
    except Exception as e:
        print(f"이동 실패: {patch}, 이유: {e}")

print("패치 이동 및 정리 완료!")


이동 완료: O09-03495_10436_15305.png
이동 완료: O09-03495_10436_15817.png
이동 완료: O09-03495_10436_16329.png
이동 완료: O09-03495_10436_16841.png
이동 완료: O09-03495_10436_17353.png
이동 완료: O09-03495_10436_17865.png
이동 완료: O09-03495_10436_19913.png
이동 완료: O09-03495_10436_20425.png
이동 완료: O09-03495_10436_20937.png
이동 완료: O09-03495_10436_21449.png
이동 완료: O09-03495_10436_7625.png
이동 완료: O09-03495_10436_8137.png
이동 완료: O09-03495_10436_8649.png
이동 완료: O09-03495_10436_9161.png
이동 완료: O09-03495_10948_14793.png
이동 완료: O09-03495_10948_15305.png
이동 완료: O09-03495_10948_15817.png
이동 완료: O09-03495_10948_16329.png
이동 완료: O09-03495_10948_20937.png
이동 완료: O09-03495_10948_21449.png
이동 완료: O09-03495_10948_21961.png
이동 완료: O09-03495_10948_22473.png
이동 완료: O09-03495_10948_7113.png
이동 완료: O09-03495_10948_7625.png
이동 완료: O09-03495_10948_8137.png
이동 완료: O09-03495_10948_8649.png
이동 완료: O09-03495_11460_14281.png
이동 완료: O09-03495_11460_21961.png
이동 완료: O09-03495_11460_22473.png
이동 완료: O09-03495_11460_22985.png
이동 완료: O09-03495_1